In [1]:
import requests
import csv
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen

In [2]:
import re
TAG_RE = re.compile(r'<[^>]+>')
def remove_tags(text):
    return TAG_RE.sub('', text)

In [3]:
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

In [4]:
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs= True
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [5]:
# Hyperparameters
amazon_reviews_count = 3

In [6]:
def book_details_goodreads(isbn):

    # Getting book title from Amazon
    amazon_base_url = "https://www.amazon.com/dp/"
    amazon_url = amazon_base_url + isbn
    req = Request(amazon_url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read().decode("utf-8")
    soup = BeautifulSoup(page, 'html.parser')

    a_title = soup.find_all("span",id="productTitle")
    a_title = str(a_title)
    a_title = remove_tags(a_title)
    a_title = a_title.strip("[]")
    a_title = a_title.strip()
    a_title_list = a_title.split(" ")

    # Goodreads Scraping
    goodreads_base_url = "https://www.goodreads.com/book/isbn/"
    goodreads_url = goodreads_base_url + isbn
    req = Request(goodreads_url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read().decode("utf-8")
    soup = BeautifulSoup(page, 'html.parser')

    # Book Title
    book_name = soup.find(itemprop="name")
    book_name = str(book_name)
    book_name = remove_tags(book_name)
    book_name = book_name.strip()
    book_name_list = book_name.split(" ")

    # Verifying if isbn is present in Goodreads by comparing book titles from Goodreads & Amazon
#     if str(a_title_list[0]).lower() != str(book_name_list[0]).lower():
#         return print("Book not found in Goodreads")

    # Author Names
    author_names = soup.find_all("span",itemprop="name")
    author_names = str(author_names)
    author_names = author_names.split(",")
    author_name = author_names[0]
    author_name = author_name.split(">")[1].split("<")[0]
    for i in range(len(author_names)):
        author_names[i] = author_names[i].split(">")[1].split("<")[0]

    author_names_text = ""
    for i in range(len(author_names)):
        author_names_text += str(author_names[i])
        author_names_text += ", "

    # Number of Ratings
    rating_count = soup.find(itemprop="ratingCount")
    rating_count = str(rating_count)
    rating_count = rating_count.split('"')[1]

    # Average Rating
    rating_val = soup.find(itemprop="ratingValue")
    rating_val = str(rating_val)
    rating_val = remove_tags(rating_val)

    # Number of pages in book
    pg_count = soup.find("meta",  property="books:page_count")
    pg_count = str(pg_count)
    pg_count = pg_count.split('"')[1]

    # Book Description
    desc = soup.find("div", id="description")
    if desc is not None:
        desc = desc.find_all("span",style="display:none")
        if desc is not None:
            desc = str(desc)
            desc = remove_tags(desc)
            description = desc.strip("[]")
            description = description.strip()
        else:
            description = "No description found"
    else:
        description = "No description found"
        
        
    # Reviews
    review_block = soup.find('div', {'id': 'reviews'})
    reviews = review_block.find_all('span', {'id': re.compile('^freeTextContainer')})
    actual_reviews = []
    for review in reviews:
        review = strip_tags(str(review))
        end_index = review.rfind(".")
        review = review[:end_index + 1]
        actual_reviews.append(review)
        
    # Friend reviews
    review_block = soup.find('div', {'id': 'reviews'})
    reviews = review_block.find_all('span', {'id': re.compile('^freeTextContainer')})
    actual_reviews = []
    for review in reviews:
        review = strip_tags(str(review))
        end_index = review.rfind(".")
        review = review[:end_index + 1]
        actual_reviews.append(review)

    # Printing book details from Goodreads
    printmd('**Book Details from Goodreads\n**')
    #print("Book Details from Goodreads\n")
    print("Book Title: ",book_name.splitlines()[0])
    #print("\n")
    print("Authors: ",author_names_text)
    #print("\n")
    print("Average Rating: ",rating_val)
    #print("\n")
    print("Number of ratings: ",rating_count)
    #print("\n")
    print("Number of pages in book: ",pg_count)
    print("\n")
    print("Book Description:")
    print("\n")
    print(description)
    print("Book Reviews:")
    print(actual_reviews)
    
    

In [28]:
def book_details_amazon(isbn):

    # Amazon Scraping
    amazon_base_url = "https://www.amazon.com/dp/"
    amazon_url = amazon_base_url + isbn
    req = Request(amazon_url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read().decode("utf-8")
    soup = BeautifulSoup(page, 'html.parser')

    # Book title
    a_title = soup.find_all("span",id="productTitle")
    a_title = str(a_title)
    a_title = remove_tags(a_title)
    a_title = a_title.strip("[]")
    a_title = a_title.strip()

    # Book details
    book_info = []
    for li in soup.select('table#productDetailsTable div.content ul li'):
        try:
            title = li.b
            key = title.text.strip().rstrip(':')
            value = title.next_sibling.strip()
            value = value.strip("()")
            book_info.append((key,value))
        except AttributeError:
            break

    # Amazon reviews scraping
    amazon_review_base_url = "https://www.amazon.com/product-reviews/"
    amazon_review_url = amazon_review_base_url + isbn + "/ref=cm_cr_getr_d_paging_btm_2?pageNumber="
    print(amazon_review_url)
    req = Request(amazon_review_url, headers={'User-Agent': 'Mozilla/5.0'})
    page = urlopen(req).read().decode("utf-8")
    soup = BeautifulSoup(page, 'html.parser')
    txt = soup.find('div', {'id': 'cm_cr-review_list'})
    print(txt)
    

    # List of book reviews in Amazon
    reviews_list = []
    reviews_list_final = []
    
    for pg in range(1):
#         amazon_review_url = amazon_review_base_url + isbn + "/ref=cm_cr_getr_d_paging_btm_2?pageNumber=" + str(pg)
#         req = Request(amazon_review_url, headers={'User-Agent': 'Mozilla/5.0'})
#         page = urlopen(req).read().decode("utf-8")
#         soup = BeautifulSoup(page, 'html.parser')
#         print(soup)
        txt = soup.find("div", id='cm_cr-review_list')

#         txt = soup.find("div", id=re.compile('^cm_cr-review_list'))
        print("txt: ")
        print(len(txt))
        try:
            for rawreview in txt.find_all('span', {'class' : 'a-size-base review-text'}):
                text = rawreview.parent.parent.parent.text
                startindex = text.index('5 stars') + 7
                endindex = text.index('Was this review helpful to you?')
                text = text[startindex:endindex]
                text = text.split("Verified Purchase")[1]
                rText = text.split(".")[:-1]
                review_text = ""
                for i in range(len(rText)):
                    review_text += rText[i]
                    review_text += "."
                if review_text is not "":
                    if "|" not in review_text:
                        reviews_list.append(review_text)
                    else:
                        rText = text.split(".")[:-2]
                        review_text = ""
                        for x in range(len(rText)):
                            review_text += rText[x]
                            review_text += "."
                        reviews_list.append(review_text)
        except AttributeError:
            review_text = "No reviews found."

    print(len(reviews_list))
    if amazon_reviews_count < len(reviews_list):
        reviews_list_final = reviews_list[:amazon_reviews_count]
    else:
        reviews_list_final = reviews_list

    # Printing book details from Amazon
    printmd('**Book Details from Amazon\n**')
    #print("Book Details from Amazon\n")
    print("Book Title: ",a_title)
    #print("\n")
    for i in range(len(book_info)):
        print(f"{book_info[i][0]} : {book_info[i][1]}")
        #print("\n")
    print("\n")
    if len(reviews_list_final) == 0:
        print(review_text)
        print("\n")
    else:
        print(f"Displaying top {amazon_reviews_count} book reviews:\n")
        for i in range(len(reviews_list_final)):
            review_txt_list = reviews_list_final[i].split(".")[:3]
            review_txt = ""
            for j in range(len(review_txt_list)):
                review_txt += review_txt_list[j]
                review_txt += "."
            review_txt += ".."
            print(review_txt)
            print("\n")

In [29]:
isbn = "1476779112"
print("hello world")

hello world


In [30]:
book_details_goodreads(isbn)

**Book Details from Goodreads
**

Book Title:  Uncertain Logic
Authors:  Christopher L. Bennett, 
Average Rating:  
  4.10

Number of ratings:  404
Number of pages in book:  384


Book Description:


An original novel continuing the saga of the TV series Star Trek: Enterprise—featuring Captain Jonathan Archer and the crew of the Enterprise!Years ago, Jonathan Archer and T’Pol helped unearth the true writings of Vulcan’s great philosopher Surak, bringing forth a new era of peaceful reform on Vulcan. But when their discovery is seemingly proven to be a fraud, the scandal threatens to undo a decade of progress and return power to the old, warlike regime. Admiral Archer, Captain T’Pol, and the crew of the U.S.S. Endeavour investigate with help from their Vulcan allies, but none of them suspect the identity of the real mastermind behind the conspiracy to reconquer Vulcan—or the price they will have to pay to discover the truth.Meanwhile, when a long-forgotten technological threat re-emerges beyond the Federation’s borders, 

  book_details_amazon(isbn)

In [31]:
  book_details_amazon(isbn)

https://www.amazon.com/product-reviews/1476779112/ref=cm_cr_getr_d_paging_btm_2?pageNumber=
None
txt: 


TypeError: object of type 'NoneType' has no len()